In [5]:
using CairoMakie, QuantEcon, LinearAlgebra, Optim, Dierckx

In [6]:
include("crra_code.jl")
include("ces_code.jl")

hpi_CES (generic function with 1 method)

In [7]:
function invariant_distribution(M, O, X, Y, Inv, policy, Π, Avals, n, pars)
    (; na, nz) = pars
    for i in 1:na
        for j in 1:nz
            if policy[i,j] <+ Avals[1]
                M[i,j,1,:] = Π[j,:]
            elseif policy[i,j] >= Avals[end]
                M[i,j,end,:] = Π[j,:]
            else
                index = findfirst(x -> x > policy[i,j], Avals)
                π = (Avals[index] - policy[i,j]) / (Avals[index] - Avals[index-1])
                M[i,j,index-1,:] = π * Π[j,:]
                M[i,j,index,:] = (1-π) * Π[j,:]
            end
        end
    end
    O = reshape(M, n, n)
    for i in 1:na
        O[i,:] = O[i,:] / sum(O[i,:])
    end
    X = reduce(vcat, [I(n) - O', ones(n)])
    Y = [zeros(n);1]
    Inv = lstsq(X, Y).solution
    Inv[Inv < 0] .= 0
    Inv = Inv / sum(Inv)
    return Inv
end

invariant_distribution (generic function with 1 method)

In [ ]:
function equilibrium(pars)
    (; na, nz) = pars
    n = na * nz
    M = zeros(na, nz, na, nz)
    O = zeros(n, n)
    X = zeros(n+1,n)
    Y = zeros(n+1)
    Inv = zeros(n)